In [ ]:
#library
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from spacy.tokens import DocBin
import json
from datetime import datetime
from tqdm import tqdm
import re


# DEFINE  functions

In [6]:
def find_intent_index(input_embed, intents_embed):
  vector = input_embed
  cosine_score = cosine_similarity([vector], intents_embed)

  intent_index = np.argmax(cosine_score, axis = 1)[0]
  print('INTENTS SENTENCE:',intents[intent_index])

  return intent_index


def match_intent_to_function(intent_index): #intent_index = index of the prebuild intents
  if intent_index == 0:
    return 'create_event'
    return
  elif intent_index == 1:
    return 'delete_event'   #_find_entities(input_sentence1)

  elif intent_index == 2:
    return 'get_calendar_events' #_find_entities(input_sentence1)

  elif intent_index == 3:
    return 'get_free_busy_info'

  elif intent_index == 4:
    return 'update_event'

  elif intent_index == 5:
    return 'reschedule_event'

  else:
    return 'NA'


In [7]:
def find_necessary_keywords(function_name):

  if function_name == 'create_event': #summary, start_time, end_time, location=None
    arguments_list = ['summary', 'start_time', 'end_time', 'location']

  if function_name == 'delete_event': #event_id
    arguments_list = ['event_id']

  if function_name == 'get_calendar_events': # date
    arguments_list = ['date']

  if function_name == 'get_free_busy_info': #(start_time, end_time)
    arguments_list = ['start_time', 'end_time']

  if function_name == 'update_event': #event_id, summary=None, start_time=None, end_time=None, location=None):
    arguments_list = ['event_id', 'summary', 'start_time', 'end_time', 'location']

  if function_name == 'reschedule_event': #(event_id, start_time, end_time)
    arguments_list = ['event_id', 'start_time', 'end_time']

  upper_arguments_list = [x.upper() for x in arguments_list]
  return upper_arguments_list


def required_arguments(function_name):
  if function_name == 'create_event': #summary, start_time, end_time, location=None
    required_list = ['summary', 'start_time', 'end_time']

  if function_name == 'delete_event': #event_id
    required_list = ['event_id']

  if function_name == 'get_calendar_events': # date
    required_list = ['date']

  if function_name == 'get_free_busy_info': #(start_time, end_time)
    required_list = ['start_time', 'end_time']

  if function_name == 'update_event': #event_id, summary=None, start_time=None, end_time=None, location=None):
    required_list = ['event_id']

  if function_name == 'reschedule_event': #(event_id, start_time, end_time)
    required_list = ['event_id', 'start_time', 'end_time']

  upper_required_list = [x.upper() for x in required_list]
  return upper_required_list


In [8]:
def return_arguments(arguments_list, doc):
  '''tagged sentence with customized NER, and extract nessary arguments (based on the function called) frrom tagging '''
  arguments_dict = {}

  for ent in doc.ents:
    if ent.label_ in (arguments_list):
      if ent.label_ not in arguments_dict.keys(): # will keep the first key
        arguments_dict.update({ent.label_: ent.text})  # if delete 'if' statement, will overwrite and keep the last

    #print(arguments_dict)

  return arguments_dict #arguments_dict['DATE'] is 'Landscapes LLC'

In [9]:
# temp demo for 1 google function

def check_info_requirement(function_name, arguments_list_need, arg_dict_true):
  response_arguments = ''
  missing_key = []
  flag_add_new_info = False # GLOBAL

  #check if find everything required: (only location is optional)
  response_function = 'You want to {}. '.format(function_name)

  for key in arguments_list_need:
    #key = arg_dict_true[i]
    if key in arg_dict_true.keys():
      value_in_key = arg_dict_true[key] #maybe empty
      response_arguments += 'the {} = {} '.format(key, value_in_key)

    else: # needed key not in true keys
    #if is NOT optional key (required key -> based on function)
      if key in required_arguments(function_name): # call the function, return a list ## need modified
        missing_key.append(key)

  if len(missing_key) == 0: #all required
    #flag_add_new_info = False
    response = response_function + response_arguments + " All requirments are satisfied."
  else:
    flag_add_new_info = True
    response_need_more_info = ' We need more information on {}, please add them.'.format(missing_key)
    response = response_function + response_need_more_info


  return response



# Pipline / write it in Class?

In [3]:
# prepare GLOBAL variables
model_pretrained = 'bert-base-nli-mean-tokens'
model = SentenceTransformer(model_pretrained)

ner_model_path = 'drive/MyDrive/capstone_ner_output' # need to update NER model
ner_output = spacy.load(ner_model_path + "/model-best")

intents = [
    "create a new event", #0
    "delete a certain event",#1
    "retrieve event detail of a certain date", #2 -> TAG: retrieving event (intent) today (entity arguments)
    "get free time information during certain time period", #3
    "change the location or summary of an event",#4
    "reschedule the time of an event" #5
    ]

intents_embed_given = model.encode(intents)



In [4]:
def intent_entity_recognition(input_sentences):

  input_embed_1 = model.encode(input_sentences)

  idx_1 = find_intent_index(input_embed = input_embed_1, intents_embed =intents_embed_given ) #intents_embed is given


  # get the function name. e.g. 'create_event'
  function_name_1 = match_intent_to_function(intent_index = idx_1)

  # get the needed arguments
  arguments_list_1 = find_necessary_keywords(function_name = function_name_1)
  doc_1 = ner_output(input_sentences) #tagged NER

  arg_dict = return_arguments(arguments_list = arguments_list_1, doc = doc_1)

  print('the function you want to call is {}, the keywords are {}'.format(function_name_1, arg_dict))

  reply_the_sentence = check_info_requirement(function_name = function_name_1,
                                              arguments_list_need = arguments_list_1,
                                              arg_dict_true = arg_dict)

  return (function_name_1, arg_dict, reply_the_sentence)

# TEST IT

In [11]:

intent_entity_recognition(input_sentences = "I want to get the mondays event starting at 9am and until 5pm. ")

INTENTS SENTENCE: reschedule the time of an event
the function you want to call is reschedule_event, the keywords are {'START_TIME': '9am', 'END_TIME': 'until 5pm'}


('reschedule_event',
 {'START_TIME': '9am', 'END_TIME': 'until 5pm'},
 "You want to reschedule_event.  We need more information on ['EVENT_ID'], please add them.")

In [13]:
intent_entity_recognition(input_sentences = '''schedule a client meeting on December 10, 2023, starting at 10:30 AM. \
The meeting is expected to last for about 2 hours. We will meet at their office location. The event ID is z0gpmm1huzxw75ar0kiedcoda''')

INTENTS SENTENCE: reschedule the time of an event
the function you want to call is reschedule_event, the keywords are {'START_TIME': '10:30 AM', 'END_TIME': 'last for about 2 hours', 'EVENT_ID': 'z0gpmm1huzxw75ar0kiedcoda'}


('reschedule_event',
 {'START_TIME': '10:30 AM',
  'END_TIME': 'last for about 2 hours',
  'EVENT_ID': 'z0gpmm1huzxw75ar0kiedcoda'},
 'You want to reschedule_event. the EVENT_ID = z0gpmm1huzxw75ar0kiedcoda the START_TIME = 10:30 AM the END_TIME = last for about 2 hours  All requirments are satisfied.')